In [1]:
# crap version

import scipy.io 
import numpy as np
from numpy import linalg as LA
import matplotlib
import time
from PIL import Image

function for viewing images

In [2]:
def show_img(img):
    # shows image as png
    # img must be a 2576 1d vector
    new_img = img.copy()
    new_img.resize((46,56))
    im = Image.fromarray(np.transpose(new_img))
    im.show()

function for normalising zero centered images for viewing

In [3]:
def norm_image(img):
    new_img = img.copy() / np.max(img)
    new_img = new_img * np.array([128]) + np.array([128])
    return new_img

function for measuring mean squared error

In [4]:
def get_err(x, y):
    # they must be the same dimension
    err = x - y
    show_img(err)
    err = np.square(err)
    return np.sum(err)

Load the relevant data into .mat into a python dictionary

In [5]:
mat = scipy.io.loadmat('face.mat')
raw_data = np.transpose(mat['X'])

create training and test data

In [6]:
training_data = []
# np.empty([int(520*0.8), 2576])
testing_data = []
# np.empty([int(520*0.2), 2576])

for x in range(52):
        # 8/2 ratio for training and testing datasets	
	training_data[x*8:(x+1)*8] = raw_data[x*10:x*10+8]
	testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]
training_data = np.array(training_data)
testing_data = np.array(testing_data)

In [7]:
# show_img(training_data[0])
print(len(training_data))

416


Get deviation from the mean face

In [8]:
mean_train_face = np.mean(training_data, axis=0)
mean_test_face = np.mean(testing_data, axis=0)
diff_train_face = training_data - mean_train_face.T
diff_test_face = testing_data - mean_test_face.T

In [9]:
print(testing_data[0])

[ 96  93  95 ... 100  85 121]


In [10]:
print(diff_train_face[0])

[-16.72355769 -19.24278846 -17.34134615 ... -47.81490385 -46.81009615
 -40.23798077]


compute the covariance matrix for low-dimensional computation

In [11]:
start = time.time()
low_cov_mat = np.dot(diff_train_face.T, diff_train_face)
# low_cov_mat = np.dot(diff_train_face, diff_train_face.T)

low_cov_mat /= np.max(low_cov_mat)
end = time.time()
print("time required to compute low-dim-cov-mat : ",end - start)
print("shape of cov_mat: ", low_cov_mat.shape)

time required to compute low-dim-cov-mat :  0.14151930809020996
shape of cov_mat:  (2576, 2576)


In [12]:
print(low_cov_mat)

[[ 0.57005225  0.53605848  0.48574942 ... -0.12233692 -0.1384704
  -0.13520237]
 [ 0.53605848  0.56792967  0.53598011 ... -0.11638623 -0.13293959
  -0.13001596]
 [ 0.48574942  0.53598011  0.57436471 ... -0.11813704 -0.13194724
  -0.12727457]
 ...
 [-0.12233692 -0.11638623 -0.11813704 ...  0.68786948  0.66039597
   0.62514444]
 [-0.1384704  -0.13293959 -0.13194724 ...  0.66039597  0.68940666
   0.66300804]
 [-0.13520237 -0.13001596 -0.12727457 ...  0.62514444  0.66300804
   0.70116339]]


compute the eigenvalues and eigenvectors of the covariance matrix
w -> eigenvalues
v -> eigenvectors

In [22]:
start = time.time()
low_w, low_v = LA.eig(low_cov_mat)
# reconstruct original eignevectors
# do we need to normalise
# low_computed_u = np.dot(training_data.T,low_v).T
low_computed_u = low_v
# normalise for eigen vec for finding weights
for x in low_computed_u:
    x /= LA.norm(x, ord=2, axis=0)
end = time.time()
print("time required to compute eigenvalues and eigenvectors of low-dim-cov-mat : ",end-start)


print(low_computed_u.shape)

lowdim_eigenvec = low_computed_u[low_w != 0]
print(lowdim_eigenvec == low_computed_u)
print(lowdim_eigenvec.shape)

time required to compute eigenvalues and eigenvectors of low-dim-cov-mat :  6.195406913757324
(2576, 2576)
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
(2576, 2576)


Calculate weights associated with each eigenvec

In [14]:
lowdim_weights = np.dot(lowdim_eigenvec, training_data.T).T

print(lowdim_weights.shape)

(416, 2576)


In [21]:
print(training_data[0])
print(lowdim_eigenvec.shape)
print(training_data.shape)
print(low_computed_u[1])
print(training_data[0])

[106 103 103 ...  37  36  41]
(2576, 2576)
(416, 2576)
[-0.02504856+0.j        -0.03623655+0.j        -0.02255595+0.j        ...
 -0.00098333+0.0002985j -0.00098333-0.0002985j -0.00016827+0.j       ]
[106 103 103 ...  37  36  41]


In [16]:
print(lowdim_weights.shape)

(416, 2576)


In [17]:
print(lowdim_eigenvec.shape)

(2576, 2576)


Reconstruct differences from mean

In [18]:
crappyfaces = np.dot(lowdim_eigenvec.T, lowdim_weights.T).T

In [19]:
print(crappyfaces.shape)

(416, 2576)


In [20]:
x = crappyfaces[14] - mean_train_face
y = training_data[14]
print(get_err(x, y))
show_img(x)
show_img(y)

TypeError: Cannot handle this data type

In [ ]:
# for x in nz_eigenvectors[0:20]:
#     show_img(x)

sort form largest to smallest eigen vector

compute the covariance matrix for normal PCA computation

In [ ]:
normal_cov_mat = np.cov(np.transpose(diff_train_face)) # need timing
print("Length of normal cov matrix", len(normal_cov_mat))

compute eigenvalues and vectors for normal PCA computation
normal_t -> eigenvalues
normal_u -> eigenvectors

In [ ]:
normal_t, normal_u = LA.eig(normal_cov_mat) # need to do timing

get the eigen vectors corresponding to the largest 416 eigen values

In [ ]:
normal_eigenval_vec = [(x,y) for x, y in zip(normal_t, normal_u)]
sorted_normal = sorted(normal_eigenval_vec, key=lambda eigenv: eigenv[0], reverse=True)
sorted_normal = sorted_normal[0:416]

check that the eigen values and the eigen vectors are the same from both computations

In [ ]:
print("are the computed eigen values and vectors the same?")
print("size of sorted", len)
if np.array_equal(sorted_normal, sorted_low):
	print("yoyoyo")
else:
	print("f off")